In [47]:
import sys
print (sys.version)
use_sql = True
small_set = True
filename = "c:/progs/logs/quotes.csv"
print(__doc__)

# Author: Gael Varoquaux gael.varoquaux@normalesup.org
# License: BSD 3 clause

from datetime import datetime

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from six.moves.urllib.request import urlopen
from six.moves.urllib.parse import urlencode
from sklearn import cluster, covariance, manifold
import psycopg2
import pprint
import pandas as pd
import sys,os
import math

symbol_dict = {
     'TOT': 'Total',
    'XOM': 'Exxon',
    'CVX': 'Chevron',
    'COP': 'ConocoPhillips',
    'VLO': 'Valero Energy',
    'MSFT': 'Microsoft',
    'IBM': 'IBM',
    'TWX': 'Time Warner',
    'CMCSA': 'Comcast',
    'CVC': 'Cablevision',
    'YHOO': 'Yahoo',
    'DELL': 'Dell',
    'HPQ': 'HP',
    'AMZN': 'Amazon',
    'TM': 'Toyota',
    'CAJ': 'Canon',
    'SNE': 'Sony',
    'F': 'Ford',
    'HMC': 'Honda',
    'NAV': 'Navistar',
    'NOC': 'Northrop Grumman',
    'BA': 'Boeing',
    'KO': 'Coca Cola',
    'MMM': '3M',
    'MCD': 'McDonald\'s',
    'PEP': 'Pepsi',
    'K': 'Kellogg',
    'UN': 'Unilever',
    'MAR': 'Marriott',
    'PG': 'Procter Gamble',
    'CL': 'Colgate-Palmolive',
    'GE': 'General Electrics',
    'WFC': 'Wells Fargo',
    'JPM': 'JPMorgan Chase',
    'AIG': 'AIG',
    'AXP': 'American express',
    'BAC': 'Bank of America',
    'GS': 'Goldman Sachs',
    'AAPL': 'Apple',
    'SAP': 'SAP',
    'CSCO': 'Cisco',
    'TXN': 'Texas Instruments',
    'XRX': 'Xerox',
    'WMT': 'Wal-Mart',
    'HD': 'Home Depot',
    'GSK': 'GlaxoSmithKline',
    'PFE': 'Pfizer',
    'SNY': 'Sanofi-Aventis',
    'NVS': 'Novartis',
    'KMB': 'Kimberly-Clark',
    'R': 'Ryder',
    'GD': 'General Dynamics',
    'RTN': 'Raytheon',
    'CVS': 'CVS',
    'CAT': 'Caterpillar',
    'DD': 'DuPont de Nemours'}

if(use_sql):
    conn = psycopg2.connect("dbname=%s user=%s host='localhost' password=%s" % ("financedb_dev",os.environ['finance_dev_usr'],os.environ['finance_dev_pwd']))

columns = ['ticker','date','open','close','high','low','volume']
if use_sql: 
    vals = ', '.join(['ticker','date','open*adjustment_factor as open','close*adjustment_factor as close','high*adjustment_factor as high','low*adjustment_factor as low','volume'])
    query = "select %s from quote where ticker in ('%s') and date > '%s'"  % (vals,"', '".join(symbol_dict.keys()), "2009-01-01")
    print(query)

    quotes = pd.read_sql(query, conn)
else:
    quotes = pd.read_csv(filename)


3.6.0 |Anaconda custom (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]
Automatically created module for IPython interactive environment
select ticker, date, open*adjustment_factor as open, close*adjustment_factor as close, high*adjustment_factor as high, low*adjustment_factor as low, volume from quote where ticker in ('TOT', 'XOM', 'CVX', 'COP', 'VLO', 'MSFT', 'IBM', 'TWX', 'CMCSA', 'CVC', 'YHOO', 'DELL', 'HPQ', 'AMZN', 'TM', 'CAJ', 'SNE', 'F', 'HMC', 'NAV', 'NOC', 'BA', 'KO', 'MMM', 'MCD', 'PEP', 'K', 'UN', 'MAR', 'PG', 'CL', 'GE', 'WFC', 'JPM', 'AIG', 'AXP', 'BAC', 'GS', 'AAPL', 'SAP', 'CSCO', 'TXN', 'XRX', 'WMT', 'HD', 'GSK', 'PFE', 'SNY', 'NVS', 'KMB', 'R', 'GD', 'RTN', 'CVS', 'CAT', 'DD') and date > '2009-01-01'


In [43]:
q = quotes

In [44]:
q.head()

,ticker,date,open,close,high,low,volume
0,COP,2009-01-02,21.764451,30.126780,23.129440,21.701646,15538200
1,COP,2009-01-05,22.882392,30.467306,23.468584,22.798651,20390200
2,COP,2009-01-06,23.669572,30.582661,24.050597,23.158750,17082100
3,COP,2009-01-07,23.028949,29.242476,23.028949,22.082669,13447100
4,COP,2009-01-08,22.187346,29.654419,22.710732,22.011490,10084300


In [45]:
q.groupby('ticker').agg({'low':['min'],'high':['max'],'volume':['sum'] })

,low,high,volume
,min,max,sum
ticker,,,
COP,14.446956,79.209830,22125504601
CVX,41.401469,119.194712,17819357268
IBM,66.770894,190.314575,11097569760
MSFT,12.010253,74.379997,99818503417
TOT,26.308427,62.754945,5288935689
TWX,12.591222,102.449997,14589149644
VLO,10.512435,69.914548,19621953861
XOM,45.629843,94.688667,37664020132


In [46]:
q.groupby('ticker').apply(lambda x:x.open*x['volume']).groupby('ticker').agg('sum')

ticker
COP     8.220727e+11
CVX     1.441276e+12
IBM     1.514766e+12
MSFT    3.016850e+12
TOT     2.087668e+11
TWX     6.496498e+11
VLO     5.658629e+11
XOM     2.576578e+12
dtype: float64